<a href="https://colab.research.google.com/github/roshantac/EVA4/blob/master/Assignment15/A/Depthmap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 10859843474004450651, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 5572905737203118132
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 10784727761128795725
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15701463552
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 14150770272096245306
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

In [18]:
!nvidia-smi

Fri May  8 19:37:30 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    32W / 250W |  15523MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
!git clone https://github.com/roshantac/DenseDepth.git

Cloning into 'DenseDepth'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 47 (delta 5), reused 47 (delta 5), pack-reused 0
Unpacking objects: 100% (47/47), done.


In [3]:
!wget https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5 -O ./DenseDepth/nyu.h5

--2020-05-08 19:21:47--  https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.96.42
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.96.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 172897376 (165M) [application/h5]
Saving to: ‘./DenseDepth/nyu.h5’

./DenseDepth/nyu.h5 100%[===================>] 164.89M  14.2MB/s    in 11s     

2020-05-08 19:21:59 (14.5 MB/s) - ‘./DenseDepth/nyu.h5’ saved [172897376/172897376]



In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!cp '/content/gdrive/My Drive/dataSet2.zip' '/content/DenseDepth/'

In [6]:
%cd DenseDepth

/content/DenseDepth


In [0]:
mkdir 'Data'

In [0]:
destDataDir = '/content/DenseDepth/Data/'
path_to_zip_file = '/content/DenseDepth/dataSet2.zip'
import zipfile
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(destDataDir)

In [0]:
rm '/content/DenseDepth/dataSet2.zip'

In [0]:

def batchSpliter(srcText,batchSize):
	fp = open(srcText,'r')
	m = fp.read().splitlines()
	strt = 0
	end = batchSize
	b= []
	for i in range(0, int(408021/batchSize)+1):
		a = m[strt:end]
		strt = end
		if((end+batchSize)<408021):
			end = end+batchSize
		else:
			end = 408021
		b.append(a)
	return b


In [0]:
!cp '/content/gdrive/My Drive/fgbgimg.txt' '/content/DenseDepth/'

In [16]:
import skimage
from skimage.transform import resize
import os
import shutil
import numpy as np
from PIL import Image
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '5'
from keras.models import load_model
from layers import BilinearUpSampling2D
from utils import predict, load_images, display_images
from matplotlib import pyplot as plt
import datetime
!cd DenseDepth/
try:
  os.mkdir("/content/DenseDepth/Data/DepthMap")
except:
  print("Either folder already exists or tree is not exist")

srcText = "/content/DenseDepth/fgbgimg.txt"
srcpath = "/content/DenseDepth/Data/Output/fgbg/"
destpath = "/content/DenseDepth/Data/DepthMap/" 
prrMdl = 'nyu.h5'
# Custom object needed for inference and training
custom_objects = {'BilinearUpSampling2D': BilinearUpSampling2D, 'depth_loss_function': None}
print('Loading model...')
idx =0
model = load_model(prrMdl, custom_objects=custom_objects, compile=False)
print('loaded ...')
for d in batchSpliter(srcText,200):
  begin_time = datetime.datetime.now()
  batch_images = []
  for dimg in d:
    imgsrc = srcpath + dimg
    print("imgsrc",imgsrc)
    img = np.clip(np.asarray((Image.open(imgsrc)).resize((640,480),Image.BILINEAR), dtype=float) / 255, 0, 1)
    batch_images.append(img)
  images = np.stack(batch_images, axis =0)
  #del batch_images
  outputs = predict(model, images)
  #del images
  print("output", outputs.shape)
  for i in range(outputs.shape[0]):
    rescaled = outputs[i][:,:,0]
    #print("rscaled ",rescaled.shape ,"  out: ",outputs[i].shape)
    rescaled = rescaled - np.min(rescaled)
    rescaled = rescaled / np.max(rescaled)
    im = Image.fromarray(255-np.uint8(rescaled*255))
    im.save(destpath + d[i], optimize=True, quality=60)
    print(idx)
    idx = idx +1
    #print(datetime.datetime.now() - begin_time)
  
  #del outputs

/bin/bash: line 0: cd: DenseDepth/: No such file or directory
Either folder already exists or tree is not exist
Loading model...
loaded ...
imgsrc /content/DenseDepth/Data/Output/fgbg/img0.jpg
imgsrc /content/DenseDepth/Data/Output/fgbg/img1.jpg
imgsrc /content/DenseDepth/Data/Output/fgbg/img2.jpg
imgsrc /content/DenseDepth/Data/Output/fgbg/img3.jpg
imgsrc /content/DenseDepth/Data/Output/fgbg/img4.jpg
imgsrc /content/DenseDepth/Data/Output/fgbg/img5.jpg
imgsrc /content/DenseDepth/Data/Output/fgbg/img6.jpg
imgsrc /content/DenseDepth/Data/Output/fgbg/img7.jpg
imgsrc /content/DenseDepth/Data/Output/fgbg/img8.jpg
imgsrc /content/DenseDepth/Data/Output/fgbg/img9.jpg
imgsrc /content/DenseDepth/Data/Output/fgbg/img10.jpg
imgsrc /content/DenseDepth/Data/Output/fgbg/img11.jpg
imgsrc /content/DenseDepth/Data/Output/fgbg/img12.jpg
imgsrc /content/DenseDepth/Data/Output/fgbg/img13.jpg
imgsrc /content/DenseDepth/Data/Output/fgbg/img14.jpg
imgsrc /content/DenseDepth/Data/Output/fgbg/img15.jpg
imgsrc

ResourceExhaustedError: ignored

In [0]:
# importing required modules 
from zipfile import ZipFile 
import os 
  
def get_all_file_paths(directory): 
  
    # initializing empty file paths list 
    file_paths = [] 
  
    # crawling through directory and subdirectories 
    for root, directories, files in os.walk(directory): 
        for filename in files: 
            # join the two strings in order to form the full filepath. 
            filepath = os.path.join(root, filename) 
            file_paths.append(filepath) 
  
    # returning all file paths 
    return file_paths         
  
def main(): 
    # path to folder which needs to be zipped 
    directory = '/content/DenseDepth/Data/DepthMap'
  
    # calling function to get all file paths in the directory 
    file_paths = get_all_file_paths(directory) 
  
    # printing the list of all files to be zipped 
    print('Following files will be zipped:') 
    for file_name in file_paths: 
        print(file_name) 
  
    # writing files to a zipfile 
    with ZipFile('depthMap.zip','w') as zip: 
        # writing each file one by one 
        for file in file_paths: 
            zip.write(file) 
  
    print('All files zipped successfully!')  
main()

he


In [0]:
!cp '/content/DenseDepth/depthMap.zip' '/content/gdrive/My Drive/'

In [1]:
!cd DenseDepth/

/content/DenseDepth


In [0]:
rm -rf /content/DenseDepth/